In [1]:
import pandas as pd
import pandasql as ps
from calc0 import *
from calc1 import *
import glob
from calc2 import *
path="C://Users//abhis//OneDrive//Desktop//excel//Timesheet//"
file_list = glob.glob(path + "//*.xlsx")

In [2]:
res=[]
for file in file_list:
    original = pd.read_excel(file)
    OTdf=original[~original['OT-In(IST)'].isna()]
    df_new = ps.sqldf('SELECT * FROM OTdf')
    df_calculate_weekend_OT = ps.sqldf('SELECT * FROM df_new WHERE Status="Weekend"')
    df_new['OT hours'] = df_new.apply(lambda x: calculateOT(str(x['OT-In(IST)'])[:8], str(x['OT-Out (IST)'])[:8]), axis=1)
    sum_of_weekend_OT = df_calculate_weekend_OT.apply(lambda x: calculateOT(str(x['OT-In(IST)'])[:8], str(x['OT-Out (IST)'])[:8]), axis=1).sum()
    sum_of_night_OT = OTdf.apply(lambda x: calculateNTOT(str(x['OT-In(IST)'])[:8], str(x['OT-Out (IST)'])[:8]), axis=1).sum()
    Sum_of_all_OT=df_new['OT hours'].sum()
    # ####  no need for this line:  Sum_of_weekend_OT=df_calculate_weekend_OT['Weekend OT hours'].sum() DO NOT UNCOMMENT THIS
    df=original.iloc[0:1,0:7]
    del df['Date']
    count_of_weekend_days=len(original[(original['Status']=='Weekend')])
    count_of_working_days=len(original[(original['Status']=='Present')])
    df['Working Days']=count_of_working_days
    df['Weekend Days']=count_of_weekend_days
    df['Total OT hours']=Sum_of_all_OT
    df['Weekend OT hours']=sum_of_weekend_OT
    df['Night OT hours']=sum_of_night_OT
    res.append(df)

In [3]:
excl_merged = pd.DataFrame()
for excl_file in res:
    excl_merged = excl_merged.append(
      excl_file, ignore_index=True)
del excl_merged['Status']

C:\Users\abhis\AppData\Local\Temp\ipykernel_668\3659478344.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  excl_merged = excl_merged.append(


In [4]:
excl_merged

# result_path="C://Users//abhis//OneDrive//Desktop//excel//Timesheet//result.xlsx"

# excl_file.to_excel(result_path)

,Employee ID,Employee Name,Domain,Client,RM/ZM,Working Days,Weekend Days,Total OT hours,Weekend OT hours,Night OT hours
0,V067,Sandeep Singh,IPO,Rakuten,Deepanshu Tewathia,23,7,27.0,NaN,0
1,V068,Sumit Kaushik,IPO,Rakuten,Deepanshu Tewathia,22,8,50.5,20.0,0


In [5]:
# result_path="C://Users//abhis//OneDrive//Desktop//excel//Timesheet//result.xlsx"
# excl_merged.to_excel(result_path)